# Running Monte Carlo Simulations on an Excel Model

Here we will see an example of how `xlwings` can be a substitute to VBA in Excel, we will run a Monte Carlo simulation on the Salary Retirement model.

In [1]:
import xlwings as xw
import pandas as pd

## The Monte Carlo Setup

First let's get a connection to our Excel sheet.

In [12]:
book = xw.Book('Dynamic Salary Retirement Model with Monte Carlo.xlsx')
sht = book.sheets['Inputs and Outputs']
sim_sht = book.sheets['Simulations']

### Running a Single Simulation

We want to evaluate how interest rate affects years until retirement. Let's first just try changing the interest rate and getting the years to retirement as the output.

In [13]:
sht.range('B10').value = 0.08

Now we can see that the interest rate has changed to 8% in Excel and that the years to retirement has changed to 24. But we want to get that output out of Excel as well.

In [14]:
years_to_retirement = sht.range('B18').value
years_to_retirement

24.0

Now that we have the value in Python we can analyze it in Python. Or if we want to analyze the results in Excel, we can output it back to the Excel workbook as a hard-coded value in a different cell, so that it will still be saved when the inputs change.

In [15]:
sht.range('E2').value = years_to_retirement

Now we can see the value is in Excel in the cell `E2`.

### Running Multiple Simulations

Just as we have done with pure Python Monte Carlo simulations, now we want to run this process many times. We'll use a loop over the number of iterations to do this. We will collect the results in Python and then output to Excel at the end.

First we need to be getting the interest rate randomly from a normal distribution:

In [16]:
import random

interest_mean = 0.05
interest_std = 0.03

interest_rate = random.normalvariate(interest_mean, interest_std)
interest_rate

0.020605745745521563

In [17]:
num_iter = 10

all_retirement_years = []
for i in range(num_iter):
    interest_rate = random.normalvariate(interest_mean, interest_std)
    sht.range('B10').value = interest_rate
    years_to_retirement = sht.range('B18').value
    all_retirement_years.append(years_to_retirement)
all_retirement_years

[26.0, 26.0, 32.0, 33.0, 26.0, 25.0, 30.0, 27.0, 21.0, 25.0]

Now output back to Excel. We want them in a column so we will do the list comprehension trick.

In [18]:
vertical_retirement_years = [[ret_year] for ret_year in all_retirement_years]
sht.range('E2').value = vertical_retirement_years

Now wrap this all up in a function.

In [19]:
def random_normal_positive(mean, std):
    value = -1
    while value < 0:
        value = random.normalvariate(mean, std)
    return value

In [20]:
def retirement_simulations(num_iter, interest_mean, interest_std):
    all_data = []
    for i in range(num_iter):
        interest_rate = random_normal_positive(interest_mean, interest_std)
        sht.range('B10').value = interest_rate
        years_to_retirement = sht.range('B18').value
        all_data.append((interest_rate, years_to_retirement))
    
    df = pd.DataFrame(all_data, columns = ['Interest', 'Years to Retirement'])
    sim_sht.range('A1').options(pd.DataFrame, index=False).value = df
    
    return df  # return it so we will also have it in Python in addition to Excel

results = retirement_simulations(1000, 0.1, 0.05)
results[:10]

,Interest,Years to Retirement
0,0.061725,27.0
1,0.192733,16.0
2,0.063810,26.0
3,0.069946,25.0
4,0.173427,17.0
5,0.079746,24.0
6,0.213477,16.0
7,0.113153,21.0
8,0.155322,18.0
9,0.164654,18.0


### Visualize and Analyze

To have a better understanding of the results, we can visualize them with:
- Histogram
- Line Chart
- Percentiles Table
- Check the Probability of a Certain Outcome
- Regression Analysis

These steps are already made on the Excel document. The Monte Carlo simulation could be run with different iterations to see some changes to the histogram, chart and percentile table. The regression analysis has been made with Excel's add-in 'Data Analysis', this needs to be done each time a model is run